In [ ]:
#Libraires import
import quandl

import pandas as pd

# Early Definitions

In [ ]:
#Quandl API key - Created a Free Account
quandl.ApiConfig.api_key = ""

In [ ]:
#Import the DataFrame with Country Names and Country ISO Codes (for Download)
countries_df = pd.read_csv('Country_ISO_code.csv')

# Obtaining the Data

In [ ]:
#This is the code for the 1st Country to be Downloaded
##Note: A DataFrame will be created with the data for the 1st country and then ...
## ... the data for the next countries will be added to this DataFrame in a loop via merger (see next cell)

#Define the Country Name and the Country ISO Code 
country_name = countries_df.iloc[0]['Country']
country_code = countries_df.iloc[0]['ISO3']

#Codes to Download the Debt level by GDP and Outstanding Debt Value using the Country ISO Code
country_debt_to_gdp = quandl.get("ODA/"+country_code+"_GGXWDG_NGDP")
country_gdp = quandl.get("ODA/"+country_code+"_NGDPD")

#Reset Index for Merging based on "Date" column
country_debt_to_gdp = country_debt_to_gdp.reset_index()
country_gdp = country_gdp.reset_index()

#Change the Name of the column with the Data from Value to the Country's Name
country_debt_to_gdp = country_debt_to_gdp.rename(columns={'Value': country_name})
country_gdp = country_gdp.rename(columns={'Value': country_name})

In [ ]:
#Loop to Obtain the data for each Country In the Countries DataFrame
##Same code as above with difference in the last 2 lines
for index, row in countries_df[1:].iterrows():
    try:
        country_name = row['Country']
        country_code = row['ISO3']

        country_debt_to_gdp_next = quandl.get("ODA/"+country_code+"_GGXWDG_NGDP")
        country_gdp_next = quandl.get("ODA/"+country_code+"_NGDPD")

        country_debt_to_gdp_next = country_debt_to_gdp_next.reset_index()
        country_gdp_next = country_gdp_next.reset_index()

        country_debt_to_gdp_next = country_debt_to_gdp_next.rename(columns={'Value': country_name})
        country_gdp_next = country_gdp_next.rename(columns={'Value': country_name})

        #Merge this with the main DataFrame
        country_debt_to_gdp = country_debt_to_gdp.merge(country_debt_to_gdp_next, on='Date', how='outer')
        country_gdp = country_gdp.merge(country_gdp_next, on='Date', how='outer')
    except:
        continue

In [ ]:
#Sort DataFrames by Date
country_debt_to_gdp = country_debt_to_gdp.sort_values(by=['Date'])
country_gdp = country_gdp.sort_values(by=['Date'])

In [ ]:
#Delete prospective rows
country_debt_to_gdp = country_debt_to_gdp.iloc[:40]
country_gdp = country_gdp.iloc[:40]

In [ ]:
#DataFrame for the Latest Value for GDP in USD Billions
country_gdp_latest = pd.DataFrame(country_gdp.iloc[-1])
country_gdp_latest = country_gdp_latest[1:]
country_gdp_latest = country_gdp_latest.reset_index()
country_gdp_latest.columns = ['Country','GDP']

#DataFrame for the Latest Value for Debt to GDP
country_debt_to_gdp_latest = pd.DataFrame(country_debt_to_gdp.iloc[-1])
country_debt_to_gdp_latest = country_debt_to_gdp_latest[1:]
country_debt_to_gdp_latest = country_debt_to_gdp_latest.reset_index()
country_debt_to_gdp_latest.columns = ['Country','Debt to GDP']
country_debt_to_gdp_latest['Rank'] = country_debt_to_gdp_latest['Debt to GDP'].rank(ascending=False).dropna().astype('int32')

In [ ]:
#DataFrame for the Latest Value for Debt in in USD Billions
country_debt_latest = country_gdp_latest.merge(country_debt_to_gdp_latest,on='Country', how='left')
country_debt_latest['Debt'] = country_debt_latest['GDP']*country_debt_latest['Debt to GDP']/100
country_debt_latest = country_debt_latest.drop(columns=['GDP','Debt to GDP', 'Rank'])

# Treemap of World Debt

In [ ]:
#Libraries for Treemap
from functools import partial
from d3IpyPlus import *
#d3IpyPlus was found in https://github.com/maclandrol/d3IpyPlus
## One just needs to include the file d3IpyPlus.py in the same path as the Jupyter Notebook's file

In [ ]:
#Running the Treemamp Code
tmap = TreeMap(id=["Country"], size="Debt", color="Debt", legend=False, width=700)
tmap.draw(country_debt_latest)

In [ ]:
#To print the dump html code the below can be used. However, in this case changes were needed in the html text.
print(tmap.dump_html(country_debt_latest))

# World Map with the Debt

In [ ]:
#Libabries for GeoPlots
import geoviews as gv
import geoviews.feature as gf
import geopandas as gpd
from cartopy import crs as ccrs

#Library to Save the Plots
import panel as pn

gv.extension('bokeh')

## Rearranging the Data

In [ ]:
##GeoPandas Database
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.rename(columns={'iso_a3':'ISO3'})
###Correcting GeoPandas Database
world.loc[world['name']=='Norway','ISO3'] = 'NOR'
world.loc[world['name']=='France','ISO3'] = 'FRA'
world.loc[world['name']=='Kosovo','ISO3'] = 'RKS'

In [ ]:
#Merge DataFrames to include ISO3 column in the DataFrame with the Debt to GDP data
country_debt_to_gdp_latest = country_debt_to_gdp_latest.merge(countries_df, on='Country', how='left')
country_debt_to_gdp_latest = country_debt_to_gdp_latest.drop(columns='Country')

#Create the DataFrame to Plot the GeooMaps: Merge the DataFrame comprising the Debt to GDP data with the DataFrame with 
#... Greo coordinates
geo_df = world.merge(country_debt_to_gdp_latest, on='ISO3', how='left')

#Reordering columns
order = [0,1,2,3,4,6,7,5] # setting column's order
geo_df = geo_df[[geo_df.columns[i] for i in order]]
geo_df = geo_df.rename(columns={'name':'Country'})

In [ ]:
##Debt to GDP GeoMap
country_debt_to_gdp_map = gv.Polygons(geo_df, vdims=['Debt to GDP','Rank','Country'],label= 'Debt to GDP (in %), 2019'
           ).opts(projection=ccrs.Robinson(), width=800, height=400,tools=['hover'], colorbar=True, cmap='RdYlGn_r')

In [ ]:
#Saving the Figures to HTML format
p = pn.panel(country_debt_to_gdp_map)
p.save('Debt_to_GDP_Map.html')

# Plotting Debt's Evolution

In [ ]:
#Libraries for Graphs
import holoviews as hv
from holoviews import opts, dim
from bokeh.models import HoverTool

hv.extension('bokeh')

In [ ]:
#Set Index for the DataFrame
#Define what are the possible entries in the DropDown List
country_debt_to_gdp = country_debt_to_gdp.set_index('Date')

countries = country_debt_to_gdp.columns

multi_df = dict()
for country in countries:

    country_table = country_debt_to_gdp[country]
    country_table = pd.DataFrame(country_table)
    country_table = country_table.reset_index()
    country_table['Date'] = pd.to_datetime(country_table['Date'])
    country_table = country_table.rename(columns={country:'Debt_to_GDP'})
    
    #Joining the obtained Single Dataframes into the Dicitonary
    multi_df.update({country: country_table})

In [ ]:
#Define the Function to be used in the Dynamic Map
def load_countries(Country):  
    #Instatiating the Curve Objects for Plotting
    hover_1 = HoverTool(tooltips=[("Date", "@Date{%F}"), ("Debt_to_GDP", "@Debt_to_GDP")], formatters={'Date': 'datetime'}) 
    
    full_graph = hv.Curve(multi_df[Country], label = 'Debt to GDP').opts(
        tools=[hover_1], xlabel='Date', ylabel = 'Debt in % of GDP', width=700, height = 400)    
    return full_graph

country_names = list(multi_df.keys())

#Instantiate the DynamicMap object
dmap = hv.DynamicMap(load_countries, kdims='Country').redim.values(Country=country_names)

dmap = dmap.opts(framewise=True)

In [ ]:
#Saving the Graph to HTML format
p = pn.panel(dmap)
p.save('Debt_to_GDP_multiple_countries.html', embed = True)